In [52]:
import pandas as pd
import numpy as np
from scipy.optimize import fsolve
from scipy.optimize import root

In [21]:
df_turbine = pd.read_csv('/Users/sushmitroy/Mathematical_statistics/Data/Turbine.csv')
# Reading few lines
wind = df_turbine['AveSpeed']

In [70]:
def weibull_shape(k,data):
    numer = sum((data**k)*np.log(data))
    denom = sum((data**k))
    return numer/denom - 1/k - np.log(data).mean()

In [78]:
root(weibull_shape, x0=1, args =(wind))['x']


array([ 3.16932037])